In [1]:
import torch 
from torch import nn
from torch.nn import functional as F
import ttach as tta
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss
import pandas as pd
import numpy as np
import torchvision
from sklearn.model_selection import KFold
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import timm
import TrainValidData
import TestData

In [2]:
labels_dataframe=pd.read_csv('data/cifar-10/trainLabels.csv')
labels_dataframe.head(5)

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [3]:
all_labels=sorted(list(set(labels_dataframe['label'])))
num_classes=len(all_labels)
print(num_classes)
all_labels

10


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
class_to_num=dict(zip(all_labels,range(num_classes)))
class_to_num

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [5]:
num_to_class={x:y for y,x in class_to_num.items()}
num_to_class

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [6]:
train_transform = torchvision.transforms.Compose([
    # 在高度和宽度上将图像放大到40像素的正方形,
    # 随机裁剪出一个高度和宽度均为40像素的正方形图像，
    # 生成一个面积为原始图像面积0.64到1倍的小正方形，
    # 然后将其缩放为高度和宽度均为32像素的正方形
    torchvision.transforms.Resize((224,224)),
        torchvision.transforms.AutoAugment(policy=torchvision.transforms.AutoAugmentPolicy.CIFAR10),
        torchvision.transforms.ToTensor(),
    # 标准化图像的每个通道
        torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

val_test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [7]:
train_img_path='data/cifar-10/train/'
train_val_path='data/cifar-10/trainLabels.csv'
test_img_path='data/cifar-10/test/'
test_path='data/cifar-10/sampleSubmission.csv'

In [8]:
def try_gpu():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
print(try_gpu())

cuda


In [9]:
!nvidia-smi

Mon Aug 16 17:28:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.63       Driver Version: 466.63       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8     6W /  N/A |    153MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
k_folds,num_epochs,learning_rate,weight_decay=5,10,1e-5,5e-4
train_loss_function=CutMixCrossEntropyLoss(True)
valid_loss_function=nn.CrossEntropyLoss()
# For fold results
results = {}
kfold=KFold(n_splits=k_folds,shuffle=True)

In [11]:
def get_model():
    model=timm.create_model('resnest50d',pretrained=True)
    model.fc=nn.Linear(model.fc.in_features,10)
    torch.nn.init.xavier_uniform_(model.fc.weight)
    return model

In [12]:
train_val_dataset = TrainValidData.TrainValidData(train_val_path, train_img_path)
for fold,(train_ids,valid_ids) in enumerate(kfold.split(train_val_dataset)):
    print('Fold',str(fold))
    print('--------------------------------------')
    train_subsampler=torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler=torch.utils.data.SubsetRandomSampler(valid_ids)

    train_loader=torch.utils.data.DataLoader(CutMix(TrainValidData.TrainValidData(train_val_path,train_img_path,train_transform), num_class=10, beta=1.0, prob=0.5, num_mix=2),
                                               sampler=train_subsampler,num_workers=3,batch_size=32)
    valid_loader=torch.utils.data.DataLoader(TrainValidData.TrainValidData(train_val_path,train_img_path,val_test_transform),
                                              batch_size=32,num_workers=3,sampler=valid_subsampler)

    model=get_model()
    model=model.to(try_gpu())
    params_1x = [param for name, param in model.named_parameters() if 'fc' not in str(name)]
    optimizer=torch.optim.AdamW([{'params':params_1x}, {'params': model.fc.parameters(), 'lr': learning_rate*10}],lr=learning_rate,weight_decay=weight_decay)
#     scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=40)
    for epoch in range(num_epochs):
        model.train()
        print('Starting epoch ',str(int(epoch+1)))
        train_losses = []
        for imags,labels in tqdm(train_loader):
            imags=imags.to(try_gpu())
            labels=labels.to(try_gpu())
            loss=train_loss_function(model(imags),labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        print('第',str(int(epoch+1)),'个epoch的学习率为：',str(optimizer.param_groups[0]['lr']))
#         scheduler.step()
        train_loss = np.sum(train_losses) / len(train_losses)
        print('[ Train ',str(epoch+1),'/',str(num_epochs),' ] loss = ',str(train_loss))
    print('Training process has finished. Saving trained model.')
    print('Starting validation')
    print('saving model with loss ',str(train_loss))
    save_path='data/cifar-10/model-fold-'+str(fold)+'.pth'
    torch.save(model.state_dict(),save_path)
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for imags,labels in tqdm(valid_loader):
            imags=imags.to(try_gpu())
            labels=labels.to(try_gpu())
            logits=model(imags)
            loss=valid_loss_function(logits,labels)
            acc = (logits.argmax(dim=-1) == labels).float().mean().cpu()
            valid_losses.append(loss.item())
            valid_accs.append(acc)
        valid_loss = np.sum(valid_losses)/len(valid_losses)
        valid_acc = np.sum(valid_accs)/len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_acc))
        print('--------------------------------------')
        results[fold] = valid_acc
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
print(f'Average: {total_summation/len(results.items())} ')

Fold 0
--------------------------------------
Starting epoch  1


  0%|                                                                                         | 0/1250 [00:00<?, ?it/s]C:\Users\user\miniconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [09:08<00:00,  2.28it/s]


第 1 个epoch的学习率为： 1e-05
[ Train  1 / 10  ] loss =  1.493860726261139
Starting epoch  2


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [09:10<00:00,  2.27it/s]


第 2 个epoch的学习率为： 1e-05
[ Train  2 / 10  ] loss =  1.1679768965721131
Starting epoch  3


...

第 9 个epoch的学习率为： 1e-05
[ Train  9 / 10  ] loss =  0.8878538999080658
Starting epoch  10


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [09:10<00:00,  2.27it/s]


第 10 个epoch的学习率为： 1e-05
[ Train  10 / 10  ] loss =  0.8812724041223526
Training process has finished. Saving trained model.
Starting validation
saving model with loss  0.8812724041223526


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:49<00:00,  6.34it/s]

[ Valid | 010/010 ] loss = 0.07104, acc = 0.97903
Accuracy for fold 4: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.9774361022364217 
Fold 1: 0.9785343450479234 
Fold 2: 0.976138178913738 
Fold 3: 0.978035143769968 
Fold 4: 0.9790335463258786 
Average: 0.977835463258786 


In [13]:
test_loader = torch.utils.data.DataLoader(TestData.TestData(test_path,test_img_path,val_test_transform),batch_size=32,num_workers=3)

In [14]:
model=get_model()
model=model.to(try_gpu())
for test_fold in range(k_folds):
    model_path='data/cifar-10/model-fold-'+str(test_fold)+'.pth'
    save_path='data/cifar-10/submission-fold-'+str(test_fold)+'.csv'
    model.load_state_dict(torch.load(model_path))
    model.eval()
#     tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))
    predictions=[]
    for imags in tqdm(test_loader):
        imags=imags.to(try_gpu())
        with torch.no_grad():
            logits=model(imags)
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    preds=[]
    for i in predictions:
        preds.append(num_to_class[i])
    test_data=pd.read_csv(test_path)
    test_data['label']=pd.Series(preds)
    submission=pd.concat([test_data['id'],test_data['label']],axis=1)
    submission.to_csv(save_path,index=False)
    print('Done!!!')

100%|██████████████████████████████████████████████████████████████████████████████| 9375/9375 [24:52<00:00,  6.28it/s]


Done!!!


100%|██████████████████████████████████████████████████████████████████████████████| 9375/9375 [20:51<00:00,  7.49it/s]


Done!!!


100%|██████████████████████████████████████████████████████████████████████████████| 9375/9375 [20:43<00:00,  7.54it/s]


Done!!!


100%|██████████████████████████████████████████████████████████████████████████████| 9375/9375 [20:45<00:00,  7.53it/s]


Done!!!


100%|██████████████████████████████████████████████████████████████████████████████| 9375/9375 [20:44<00:00,  7.53it/s]


Done!!!


In [15]:
df0=pd.read_csv('data/cifar-10/submission-fold-0.csv')
df1=pd.read_csv('data/cifar-10/submission-fold-1.csv')
df2=pd.read_csv('data/cifar-10/submission-fold-2.csv')
df3=pd.read_csv('data/cifar-10/submission-fold-3.csv')
df4=pd.read_csv('data/cifar-10/submission-fold-4.csv')


In [16]:
num_label0=[]
for i in df0['label']:
    num_label0.append(class_to_num[i])
df0['nlabel0']=num_label0
df0.head()

,id,label,nlabel0
0,1,deer,4
1,2,airplane,0
2,3,automobile,1
3,4,ship,8
4,5,airplane,0


In [17]:
num_label1=[]
for i in df1['label']:
    num_label1.append(class_to_num[i])
df1['nlabel1']=num_label1
df1.head()

,id,label,nlabel1
0,1,deer,4
1,2,airplane,0
2,3,automobile,1
3,4,ship,8
4,5,airplane,0


In [18]:
num_label2=[]
for i in df2['label']:
    num_label2.append(class_to_num[i])
df2['nlabel2']=num_label2
df2.head()

,id,label,nlabel2
0,1,deer,4
1,2,airplane,0
2,3,automobile,1
3,4,ship,8
4,5,airplane,0


In [19]:
num_label3=[]
for i in df3['label']:
    num_label3.append(class_to_num[i])
df3['nlabel3']=num_label3
df3.head()

,id,label,nlabel3
0,1,deer,4
1,2,deer,4
2,3,automobile,1
3,4,ship,8
4,5,airplane,0


In [20]:
num_label4=[]
for i in df4['label']:
    num_label4.append(class_to_num[i])
df4['nlabel4']=num_label4
df4.head()

,id,label,nlabel4
0,1,deer,4
1,2,ship,8
2,3,automobile,1
3,4,ship,8
4,5,airplane,0


In [21]:
df_all=df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,id,nlabel0
0,1,4
1,2,0
2,3,1
3,4,8
4,5,0


In [22]:
df_all['nlabel1']=df1['nlabel1']
df_all['nlabel2']=df2['nlabel2']
df_all['nlabel3']=df3['nlabel3']
df_all['nlabel4']=df4['nlabel4']
df_all.head()

,id,nlabel0,nlabel1,nlabel2,nlabel3,nlabel4
0,1,4,4,4,4,4
1,2,0,0,0,4,8
2,3,1,1,1,1,1
3,4,8,8,8,8,8
4,5,0,0,0,0,0


In [23]:
df_mode=df_all.drop(['id'],axis=1).mode(axis=1)
df_mode.head()

,0,1,2,3,4
0,4.0,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN
3,8.0,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,NaN,NaN


In [24]:
vote_class=[]
for i in df_mode[0]:
    vote_class.append(num_to_class[i])

In [25]:
df_vote=df0.copy()
df_vote=df_vote.drop(['nlabel0'],axis=1)
df_vote['label']=vote_class
df_vote.head()

,id,label
0,1,deer
1,2,airplane
2,3,automobile
3,4,ship
4,5,airplane


In [26]:
df_vote.to_csv('data/cifar-10/submission_vote.csv',index=False)